# Perform an experiment in MLFlow
- start experiment 
- train model
- save metrics
- save artifact

In [1]:
!pip install mlflow boto3 awscli -q

In [15]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

import logging

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

warnings.filterwarnings("ignore")
np.random.seed(40)

In [3]:
data = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=";")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
data.shape

(1599, 12)

In [5]:
TARGET_COLUMN = "quality"
train, test = train_test_split(data)

train_x = train.drop([TARGET_COLUMN], axis=1)
test_x = test.drop([TARGET_COLUMN], axis=1)
train_y = train[[TARGET_COLUMN]]
test_y = test[[TARGET_COLUMN]]

In [13]:
!aws --endpoint-url $MLFLOW_S3_ENDPOINT_URL s3 mb s3://mlflow

make_bucket: bpk-mlflow


In [39]:
experiment_id = mlflow.create_experiment(name="My Wine Experiments",tags={"owner": "bpk"})

MlflowException: API request to http://mlflow-server.kubeflow.svc.cluster.local:5000/api/2.0/mlflow/experiments/create failed with exception HTTPConnectionPool(host='mlflow-server.kubeflow.svc.cluster.local', port=5000): Max retries exceeded with url: /api/2.0/mlflow/experiments/create (Caused by ResponseError('too many 500 error responses'))

In [38]:
def experiment(alpha, l1_ratio):
    with mlflow.start_run(run_name='wine_models', experiment_id=experiment_id):
            mlflow.set_tag("author", "bpk")
            mlflow.log_param("alpha", alpha)
            mlflow.log_param("l1_ratio", l1_ratio)
            lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
            lr.fit(train_x, train_y)

            pred_y = lr.predict(test_x)
            mlflow.log_metric("rmse", np.sqrt(mean_squared_error(test_y, pred_y)))
            mlflow.log_metric("r2", r2_score(test_y, pred_y))
            mlflow.log_metric("mae", mean_absolute_error(test_y, pred_y))

            signature = infer_signature(test_x, pred_y)
            mlflow.sklearn.log_model(lr, "model", registered_model_name="wine-elasticnet", signature=signature)
            
experiment(0.5, 0.5)
experiment(1, 0)
experiment(0, 1)

2022/03/01 12:32:30 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under s3://mlflow/71/2eb45091f0b648a29ff1bf37823f66cd/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the tracking store. If logging to a mlflow server via REST, consider upgrading the server version to MLflow 1.7.0 or above.
Registered model 'wine-elasticnet' already exists. Creating a new version of this model...
2022/03/01 12:32:30 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: wine-elasticnet, version 5
Created version '5' of model 'wine-elasticnet'.
2022/03/01 12:33:33 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged succ